In [1]:
import pandas as pd 
import numpy as np
import csv
import baseline_functions as base

# restore saved variables
%store -r summary_drug6_ky_model

no stored variable summary_drug6_ky_model


In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test/kentucky/models/ky_fl_combined/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence']
KY_Y = KY_data['recid_drug_six_month'].values
FL_X = FL_data.loc[:,:'current_violence']
FL_Y = FL_data['recid_drug_six_month'].values

### Model

In [3]:
#### Logistic
c = [1e-4, 1e-3, 0.01, 0.1, 1]
logistic_summary = base.Logistic(KY_X, KY_Y, FL_X, FL_Y,C=c,seed=816)

#### Lasso
c = [1e-4, 1e-3, 0.01, 0.1, 1]
lasso_summary = base.Lasso(KY_X, KY_Y, FL_X, FL_Y, C=c,seed=816)

#### LinearSVM
c = [1e-4, 1e-3, 0.01, 0.1, 1]
svm_summary = base.LinearSVM(KY_X, KY_Y, FL_X, FL_Y, C=c,seed=816)

#### Random Forest
n_estimators =  [40,60]
depth = [6,7]
rf_summary = base.RF(KY_X, KY_Y, FL_X, FL_Y, 
                     depth=depth, 
                     estimators=n_estimators, 
                     seed=816)

#### XGBoost
learning_rate = [0.01]
depth = [6]
n_estimators = [200]
xgb_summary = base.XGB(KY_X, KY_Y, FL_X, FL_Y,
                       learning_rate=learning_rate, 
                       depth=depth, 
                       estimators=n_estimators,
                       seed=816)

#### save results
summary_drug6_ky_model = {"Logistic": logistic_summary,
                                    "Lasso": lasso_summary,
                                    "LinearSVM": svm_summary,
                                    "RF": rf_summary,
                                    "XGBoost": xgb_summary}
%store summary_drug6_ky_model

Stored 'summary_drug6_ky_model' (dict)


In [4]:
results = []

for model_name, model_summary in summary_drug6_ky_model.items():
    results.append([model_name, 
                    model_summary['best_auc'], 
                    model_summary['auc_diff'], 
                    model_summary['best_param'], 
                    model_summary['FL_score']])
results

[['Logistic',
  0.7258291124192487,
  0.0059812002790516905,
  {'C': 0.01},
  0.6202367284456836],
 ['Lasso',
  0.7259245061790319,
  0.0045851097700680965,
  {'C': 0.01},
  0.6095210759389864],
 ['LinearSVM',
  0.7279698288857613,
  0.004387312436222146,
  {'C': 1},
  0.6052566836148926],
 ['RF',
  0.7392930862141767,
  0.03217309867727214,
  {'max_depth': 7, 'n_estimators': 40},
  0.5571558416707671],
 ['XGBoost',
  0.740772872993565,
  0.01943135514921801,
  {'learning_rate': 0.01, 'max_depth': 6, 'n_estimators': 200},
  0.5510701984582581]]

In [5]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\KY Model\\Six Month\\"
results = [["Drug",             
            round(logistic_summary['FL_score'],3), 
            round(lasso_summary['FL_score'], 3), 
            round(svm_summary['FL_score'],3), 
            round(rf_summary['FL_score'],3), 
            round(xgb_summary['FL_score'],3)]]
with open(path + 'Six Month KY_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)